In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
def compute_iou(box1, box2):
    """
    Computes Intersection over Union (IoU) between two bounding boxes.
    """
    x1, y1, x2, y2 = box1
    x1g, y1g, x2g, y2g = box2

    # Compute the coordinates of the intersection rectangle
    xi1 = max(x1, x1g)
    yi1 = max(y1, y1g)
    xi2 = min(x2, x2g)
    yi2 = min(y2, y2g)

    # Compute the area of intersection
    inter_area = max(xi2 - xi1, 0) * max(yi2 - yi1, 0)

    # Compute the area of both bounding boxes
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2g - x1g) * (y2g - y1g)

    # Compute the union area
    union_area = box1_area + box2_area - inter_area

    # Compute IoU
    iou = inter_area / union_area if union_area > 0 else 0
    return iou

In [ ]:
def compute_ap50(pred_boxes, gt_boxes, iou_threshold=0.5):
    """
    Computes Average Precision at IoU = 0.5 (AP@50)
    """
    # Sort predicted boxes by confidence score
    pred_boxes = sorted(pred_boxes, key=lambda x: x[4], reverse=True)

    # Initialize True Positive (TP) and False Positive (FP) arrays
    tp = np.zeros(len(pred_boxes))
    fp = np.zeros(len(pred_boxes))

    # Track which ground truth boxes have been used
    detected_gt = []

    for i, pred_box in enumerate(pred_boxes):
        iou_max = 0
        matched_gt_idx = -1
        
        for j, gt_box in enumerate(gt_boxes):
            iou = compute_iou(pred_box[:4], gt_box[:4])
            if iou > iou_max:
                iou_max = iou
                matched_gt_idx = j

        # If IoU is greater than threshold and the GT box has not been detected
        if iou_max >= iou_threshold and matched_gt_idx not in detected_gt:
            tp[i] = 1
            detected_gt.append(matched_gt_idx)
        else:
            fp[i] = 1
    # Compute precision and recall
    cum_tp = np.cumsum(tp)
    cum_fp = np.cumsum(fp)
    precision = cum_tp / (cum_tp + cum_fp)
    recall = cum_tp / len(gt_boxes)

    # Compute Average Precision (AP) as the area under the precision-recall curve
    ap = np.sum(precision * np.diff(np.hstack(([0], recall))))
    return ap

In [ ]:
def compute_map50(all_predictions, all_ground_truths):
    """
    Computes the mean Average Precision at IoU=0.5 (mAP@50) for a specific class.
    """
    ap50s = []
    for pred_boxes, gt_boxes in zip(all_predictions, all_ground_truths):
        ap50 = compute_ap50(pred_boxes, gt_boxes)
        ap50s.append(ap50)
    map50 = np.mean(ap50s)
    return map50

In [ ]:
def df_to_boxes(df, is_submission=True):
    if is_submission:
        return df[['xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'class']].values.tolist()
    else:
        return df[['xmin', 'ymin', 'xmax', 'ymax', 'class']].values.tolist()

In [ ]:
def evaluate_map50(gt_df, submission_df):
    """
    Evaluates mAP@50 given ground truth and submission DataFrames.
    """
    # Group by filename or image_id
    all_predictions = []
    all_ground_truths = []
    
    for filename in tqdm(gt_df['filename'].unique(), desc='Getting prediction and ground truth boxes'):
        gt_boxes = df_to_boxes(gt_df[gt_df['filename'] == filename], is_submission=False)
        pred_boxes = df_to_boxes(submission_df[submission_df['filename'] == filename], is_submission=True)
        
        all_ground_truths.append(gt_boxes)
        all_predictions.append(pred_boxes)
    
    # Compute mAP@50
    map50 = compute_map50(all_predictions, all_ground_truths)
    return map50

In [ ]:
def main(gt_file_path, submission_file_path):
    gt = pd.read_csv(gt_file_path)
    submission = pd.read_csv(submission_file_path)
    classes = ['cow', 'goat', 'pig']
    total_map = 0
    for cls in classes:
        gt_cls_df = gt[gt['class']==cls]
        submission_cls_df = submission[submission['class']==cls].copy()
        map = evaluate_map50(gt_cls_df, submission_cls_df)
        total_map+=map
        print(f'class: {cls}\tmap: {map}')
    print(f'Average Map: {total_map/3}')